<a href="https://colab.research.google.com/github/amantayal44/ECO764-Assignment/blob/main/eco764.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import random as rand
from datetime import datetime
import sys


eps = 1
r_f = 0.03

def checkifnan(num):
	if num is np.nan:
		return 0
	else:
		return num

def calculateWeightedParam(companies,listofcompanies,x):
	totalsum = 0.0
	capsum = 0.0
	for company_name in listofcompanies:
		totalsum += companies[company_name]['CAP'][x]*companies[company_name]['PIreturn'][x]
		capsum += companies[company_name]['CAP'][x]
	if capsum == 0:
		return 0
	return (totalsum/(capsum))
	


df = pd.read_csv('France.csv')
columns = list(df.loc[2])[1:]
row_count = df.shape[0] - 5
companies = dict()
j = 0
company_name = ""
prev_err = False

date=[]
l=df.iloc[4:(4+row_count),0].tolist()
for x in range(row_count):
	date.append(str(l[x]).split()[0])


for i,col in enumerate(columns):
	idx = i + 1
	if(col == "#ERROR"):
		prev_err = True
		j = (j+1)%3
		continue
	elif(j == 0):
		company_name = col.split('-')[0].strip()
		companies[company_name] = dict()
		# CAP
		a = np.nan_to_num(np.array(df.iloc[4:(4+row_count),idx]).astype(float))
		companies[company_name]['CAP'] = a
		
	elif(j == 1):
		# ROI
		if(prev_err == True):
			company_name = col.split('-')[0].strip()
			if(company_name not in companies):
				companies[company_name] = dict()
		a = np.nan_to_num(np.array(df.iloc[4:(4+row_count),idx]).astype(float))
		companies[company_name]['ROI'] = a
	elif(j == 2):
		# PI
		if(prev_err == True):
			company_name = col.split('-')[0].strip()
			if(company_name not in companies):
				companies[company_name] = dict()
		a = np.nan_to_num(np.array(df.iloc[4:(4+row_count),idx]).astype(float))
		companies[company_name]['PI'] = a

	j = (j+1)%3
	prev_err = False
for company_name in companies:
	if 'PI' not in companies[company_name]:
		companies[company_name]['PI'] = np.array([0.]*row_count)
	if 'ROI' not in companies[company_name]:
		companies[company_name]['ROI'] = np.array([0.]*row_count)
	if 'CAP' not in companies[company_name]:
		# print(company_name)
		companies[company_name]['CAP'] = np.array([0.]*row_count)

marketcaparr = []
priceindexarr = []
growtharr = []
booktomarketarr = []
companylist =[]
smblist = []
hmllist = []
flist=[]
momlist=[]
t=[]
y_r=[]

for company_name  in companies:
	companies[company_name]['YR'] = []
	for x in range(row_count):
		companies[company_name]['YR'].append(0.0)

for x in range(0,row_count,12):
	for company_name in companies:
		# companies[company_name]['YR'] = []
		# for  iters in range(0,row_count):
		# 	companies[company_name]['YR'].append(0.0)
		for y in range(12):
			if y == 0:
				if x == 0:
					if companies[company_name]['PI'][x] > 0:
						companies[company_name]['YR'][x+y] = 100.00*(companies[company_name]['PI'][x+11]-companies[company_name]['PI'][x])/(companies[company_name]['PI'][x])
				elif x > 0:
					if companies[company_name]['PI'][x-11] > 0:
						companies[company_name]['YR'][x+y] = 100.00*(companies[company_name]['PI'][x]-companies[company_name]['PI'][x-11])/(companies[company_name]['PI'][x-11])
			elif y > 0:
				if x == 0:
					if companies[company_name]['PI'][x] > 0:
						companies[company_name]['YR'][x+y] = 100.00*(companies[company_name]['PI'][x+11]-companies[company_name]['PI'][x])/(companies[company_name]['PI'][x])
				elif x > 0:
					if companies[company_name]['PI'][x-11] > 0:
						companies[company_name]['YR'][x+y] = 100.00*(companies[company_name]['PI'][x]-companies[company_name]['PI'][x-11])/(companies[company_name]['PI'][x-11])



for company_name in companies:
	companylist.append(company_name)

three_factors = []
four_factors = []
new_mom = []
capm=[]
# print(row_count)

for x in range(row_count):
	marketcap = []
	priceindex = []
	growth = []
	booktomarket = []
	pireturnarr = []
	new_mom_arr = []

	for company_name in companies:
		if(x == 0):
			companies[company_name]['PIreturn'] = []
			companies[company_name]['PIreturn'].append(0.0)

		marketcap.append(companies[company_name]['CAP'][x])
		growth.append(companies[company_name]['ROI'][x])
		if x > 0:
			if companies[company_name]['PI'][x-1] ==0:
				pireturn=0
			else:
				pireturn = 100.00*(companies[company_name]['PI'][x]-companies[company_name]['PI'][x-1])/(companies[company_name]['PI'][x-1])
			companies[company_name]['PIreturn'].append(pireturn)
		pireturnarr.append(companies[company_name]['PIreturn'][x])
		booktomarket.append(companies[company_name]['ROI'][x])
		new_mom_arr.append(companies[company_name]['YR'][x])
	

	high = np.percentile(booktomarket,70)
	low = np.percentile(booktomarket,30)
	small = np.percentile(marketcap,30)
	big = np.percentile(marketcap,70)

	win = np.percentile(new_mom_arr,70)
	lose = np.percentile(new_mom_arr,30)

	valuefirm = []
	growthfirm = []
	neutralfirm = []
	smallfirm = []
	bigfirm = []
	winnerfirm = []
	loserfirm = []

	for y in range(len(companylist)):
		if marketcap[y] > big:
			bigfirm.append(companylist[y])
		elif marketcap[y] < small:
			smallfirm.append(companylist[y])
		if booktomarket[y] < low:
			growthfirm.append(companylist[y])
		elif booktomarket[y] > low and booktomarket[y] < high:
			neutralfirm.append(companylist[y])
		elif booktomarket[y] > high:
			valuefirm.append(companylist[y])
		if new_mom_arr[y] > win:
			winnerfirm.append(companylist[y])
		elif new_mom_arr[y] < lose:
			loserfirm.append(companylist[y])


	BV = list(set(bigfirm) & set(valuefirm))
	BN = list(set(bigfirm) & set(neutralfirm))
	BG = list(set(bigfirm) & set(growthfirm))
	SV = list(set(smallfirm) & set(valuefirm))
	SN = list(set(smallfirm) & set(neutralfirm))
	SG = list(set(smallfirm) & set(growthfirm))
	WB = list(set(winnerfirm) & set(bigfirm))
	WS = list(set(winnerfirm) & set(smallfirm))
	LB = list(set(loserfirm) & set(bigfirm))
	LS = list(set(loserfirm) & set(smallfirm))

	if len(BV)> 0:
		bvreturn = calculateWeightedParam(companies,BV,x)
	else:
		bvreturn = 0

	if len(BN)> 0:
		bnreturn = calculateWeightedParam(companies,BN,x)
	else:
		bnreturn = 0

	if len(BG)> 0:
		bgreturn = calculateWeightedParam(companies,BG,x)
	else:
		bgreturn = 0


	if len(SV)> 0:
		svreturn = calculateWeightedParam(companies,SV,x)
	else:
		svreturn = 0

	if len(SN)> 0:
		snreturn = calculateWeightedParam(companies,SN,x)
	else:
		snreturn = 0

	if len(SG)> 0:
		sgreturn = calculateWeightedParam(companies,SG,x)
	else:
		sgreturn = 0

	if len(WB)>0:
		wbreturn = calculateWeightedParam(companies,WB,x)
	else:
		wbreturn = 0

	if len(WS)>0:
		wsreturn = calculateWeightedParam(companies,WS,x)
	else:
		wsreturn = 0

	if len(LB)>0:
		lbreturn = calculateWeightedParam(companies,LB,x)
	else:
		lbreturn = 0

	if len(LS)>0:
		lsreturn = calculateWeightedParam(companies,LS,x)
	else:
		lsreturn = 0

	smb = (svreturn+snreturn+sgreturn)/3 - (bvreturn+bnreturn+bgreturn)/3 
	smblist.append(smb)

	hml = (svreturn+bvreturn)/2 - (sgreturn+bgreturn)/2
	hmllist.append(hml)

	f = calculateWeightedParam(companies,companylist,x)
	flist.append(f)

	mom = (wsreturn+wbreturn)/2 - (lsreturn+lbreturn)/2
	momlist.append(mom)

	three_factors.append([f-r_f,hml,smb])
	capm.append([f-r_f])
	four_factors.append([f-r_f,hml,smb,mom])

	t.append(x)


C = np.array(capm, np.float32)
Z = np.array(four_factors, np.float32)
X = np.array(three_factors, np.float32)

smblist[0]=1
for i in range(1,len(smblist)):
	smblist[i]=smblist[i]/100+smblist[i-1]

hmllist[0]=1
for i in range(1,len(hmllist)):
	hmllist[i]=hmllist[i]/100+hmllist[i-1]

flist[0]=1
for i in range(1,len(flist)):
	flist[i]=flist[i]/100+flist[i-1]

momlist[0]=1
for i in range(1,len(momlist)):
	momlist[i]=momlist[i]/100+momlist[i-1]

for i in range(len(date)):
	date[i] = datetime.strptime(date[i], '%m/%d/%Y')
#print(date)

i = 0
total_r2 = 0
file_w = open("FF3factor.txt", "w")
for company_name in companies:
    y = np.array(companies[company_name]['PIreturn'])
    y = y - r_f
    # print(y)
    model = LinearRegression().fit(X, y)
    r_sq = model.score(X, y)
    if i<=50 and i>0:
      total_r2 += r_sq
    file_w.write(str(i)+" & "+ company_name+" & "+str(round(model.intercept_,3))+" & "+str(round(model.coef_[0],3))+" & "+str(round(model.coef_[1],3))+" & "+str(round(model.coef_[2],3))+" \\\\ \n")
    i += 1

print("3 factor avg R2",total_r2/50)
print("Latex Table stored for FamaFench 3 factor model in FF3factor.txt ...")
file_w.close()
file_w = open("C4factor.txt", "w")
i = 0
total_r2 = 0


for company_name in companies:
    y = np.array(companies[company_name]['PIreturn'])
    y = y - r_f
    model = LinearRegression().fit(Z, y)
    r_sq = model.score(Z, y)
    if i<=50 and i>0:
      total_r2 += r_sq
    file_w.write(str(i)+" & "+ company_name+" & "+str(round(model.intercept_,3))+" & "+str(round(model.coef_[0],3))+" & "+str(round(model.coef_[1],3))+" & "+str(round(model.coef_[2],3))+" & "+str(round(model.coef_[3],3))+" \\\\ \n")
    i += 1
print("Carhart avg R2",total_r2/50)
print("Latex Table stored for Carhart 4 factor model in C4factor.txt ...")
file_w.close()
file_w = open("CAPM1factor.txt", "w")
i = 0
total_r2 = 0
for company_name in companies:
    y = np.array(companies[company_name]['PIreturn'])
    y = y - r_f
    # print(y)
    model = LinearRegression().fit(C, y)
    r_sq = model.score(C, y)
    if i<=50 and i>0:
      total_r2 += r_sq
    file_w.write(str(i)+" & "+ company_name+" & "+str(round(model.intercept_,3))+" & "+str(round(model.coef_[0],3))+" \\\\ \n")
    i += 1
print("CAPM avg R2",total_r2/50)
print("Latex Table stored for CAPM 1 factor model in CAPM1factor.txt ...")
file_w.close()



3 factor avg R2 0.3779516125603512
Latex Table stored for FamaFench 3 factor model in FF3factor.txt ...
Carhart avg R2 0.397885199154338
Latex Table stored for Carhart 4 factor model in C4factor.txt ...
CAPM avg R2 0.28147985340439535
Latex Table stored for CAPM 1 factor model in CAPM1factor.txt ...


In [ ]:
f = calculateWeightedParam(companies,companylist,x)

In [ ]:
a=np.array(companies['TOYOTA MOTOR']['PIreturn'])
np.mean(a,axis=0)


0.20338520628106413

In [ ]:
C = C.reshape((1,-1))
a = a.reshape((1,-1))
b = np.concatenate((C,a))
b.shape
np.cov(b)[0][0]

24.177527136082833

In [ ]:
file_w = open("Sharpe.txt","w")
i = 0
for company_name in companies:
  y = np.array(companies[company_name]['PIreturn'])
  y = y-r_f
  cov_matrix = np.cov(np.concatenate((y.reshape(1,-1),C.reshape(1,-1))))
  sharpe = np.mean(y)/(cov_matrix[0][0]**0.5)
  treyner_ratio = (np.mean(y)*cov_matrix[1][1])/cov_matrix[0][1]
  file_w.write(str(i)+" & "+ company_name+" & "+str(round(sharpe,3))+" & "+str(round(treyner_ratio,3))+" \\\\ \n")
  i += 1
file_w.close()

In [ ]:
for company_name in companies:
  print(company_name)

NIKKEI 225 STOCK AVERAGE
TOYOTA MOTOR
SOFTBANK GROUP
SONY
NIPPON TELG. & TEL.
FAST RETAILING
CHUGAI PHARM.
RECRUIT HOLDINGS
KDDI
SHIN
MITSUBISHI UFJ FINL.GP.
DAIICHI SANKYO
SOFTBANK
DAIKIN INDUSTRIES
TOKYO ELECTRON
FANUC
HONDA MOTOR
M3
TAKEDA PHARMACEUTICAL
DENSO
HITACHI
ITOCHU
SUMITOMO MITSUI FINL.GP.
JAPAN TOBACCO
MITSUBISHI
JAPAN POST HOLDINGS
CENTRAL JAPAN RAILWAY
KAO
MITSUBISHI ELECTRIC
MIZUHO FINL.GP.
PANASONIC
SEVEN & I HDG.
TOKIO MARINE HOLDINGS
AEON
ASTELLAS PHARMA
CANON
FUJIFILM HOLDINGS
FUJITSU
KOMATSU
MITSUI
NISSAN MOTOR
TERUMO
Z HOLDINGS
BRIDGESTONE
EAST JAPAN RAILWAY
KUBOTA
NEXON
OLYMPUS
OTSUKA HOLDINGS
SHISEIDO
EISAI
KYOCERA
MITSUBISHI ESTATE
MITSUI FUDOSAN
NTT DATA
SECOM
SUZUKI MOTOR
ASAHI GROUP HOLDINGS
BANDAI NAMCO HDG.
DAI
DAIWA HOUSE INDUSTRY
KIRIN HOLDINGS
MS&AD INSURANCE GP.HDG.
NITTO DENKO
NOMURA HDG.
OMRON
SHIONOGI
TDK
ADVANTEST
ASAHI KASEI
ENEOS HOLDINGS
KIKKOMAN
KYOWA KIRIN
NEC
RAKUTEN
SEKISUI HOUSE
SOMPO HOLDINGS
SUBARU
SUMITOMO
SUMITOMO METAL MINING
SUMITOMO